In [2]:
import pandas as pd

In [3]:
movies = pd.read_csv("Movies_relevant.csv")
movies.head()

,id,title,overview,genres,keywords,cast,crew,vote_average
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",7.2
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",6.9
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",6.3
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",7.6
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",6.1


In [4]:
movies.dropna(inplace = True)
movies.drop_duplicates(subset = 'title', inplace = True)

In [5]:
import ast
def change(x):
    L = []
    for i in ast.literal_eval(x):
        L.append(i['name'])
    return L
    
def drctr(x):
    for i in ast.literal_eval(x):
        if i['job'] == 'Director':
            return i['name']
    return ''
    
movies['genres'] = movies['genres'].apply(change)
movies['cast'] = movies['cast'].apply(lambda y: change(y)[:3])
movies['keywords'] = movies['keywords'].apply(change)
movies['director'] = movies['crew'].apply(drctr)

In [6]:
movies['overview'] = movies['overview'].apply(lambda x: x.lower().split())

In [7]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['keywords']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [9]:
movies['tags'] = movies['tags'].apply(stem)
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
vectors = tfidf.fit_transform(movies['tags']).toarray()
similarity = cosine_similarity(vectors)

In [10]:
def recommend(movie):
    movie = movie.lower()
    matches = movies[movies['title'].str.lower() == movie]
    if matches.empty:
        return "Movie not found in dataset."
    index = matches.index[0]
    dist = similarity[index]
    top = dist.argsort()[::-1][1:6]
    
    return movies['title'].iloc[top].tolist()


In [13]:
movie = input("Enter your movie: ")
recommend(movie)

['Hesher',
 'The East',
 '(500) Days of Summer',
 'Much Ado About Nothing',
 '12 Rounds']